#Import Libraries and Mount Drive


In [ ]:
import os
import sys
import shutil 
import glob

import numpy as np
import random
# import tensorflow as tf
# from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
import cv2 as cv
from skimage import transform
import skimage.io as io
from skimage.transform import resize
from skimage import transform
from skimage import img_as_ubyte

import torch
import torchvision
from time import time
from torchvision import datasets, transforms
from torchvision.io import read_image
from torch import nn, optim
from torch.utils.data import Dataset
print(torch.__version__)

1.8.1+cu101


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Create Directories to train the model

In [ ]:
parent_dir_j = "/content/drive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)"
parent_dir = "/content/drive/MyDrive/LWIR_to_RGB"
#os.mkdir(parent_dir)

In [ ]:
cd '/content/drive/MyDrive/LWIR_to_RGB'

/content/drive/.shortcut-targets-by-id/12Sz3Xm5GOveoM4c19uxdsdFReUqsU8dW/LWIR_to_RGB


In [ ]:
!ls '/content/drive/MyDrive/LWIR_to_RGB'

 A	      'checkpoints_after image aligning'      test
 B	       checkpoints_epoch400_before_aligning   train
 checkpoints   pytorch-CycleGAN-and-pix2pix	      val


In [ ]:
dirc = "A"
path_lwir = os.path.join(parent_dir,dirc)


dirc = "B"
path_rgb = os.path.join(parent_dir,dirc)

# os.mkdir(path_lwir)
# os.mkdir(path_rgb)

In [ ]:
path_lwir_train = os.path.join(path_lwir,"train" )
#os.mkdir(path_lwir_train)

path_lwir_val = os.path.join(path_lwir, "val")
#os.mkdir(path_lwir_val)

path_lwir_test = os.path.join(path_lwir, "test")
#os.mkdir(path_lwir_test)

path_rgb_train = os.path.join(path_rgb, "train")
#os.mkdir(path_rgb_train)

path_rgb_val = os.path.join(path_rgb, "val")
#os.mkdir(path_rgb_val)

path_rgb_test = os.path.join(path_rgb, "test")
#os.mkdir(path_rgb_test)


#Transfer images from dataset to train, val, test in A (LWIR) and B (RGB)


In [ ]:
def get_data(source_path,destination_path, cover_type,start_sub,end_sub,count = 1):
   
  #count = 1
  for i in range(start_sub,end_sub+1):
    if i < 10:
        data_name = "0000"+str(i)              
    else:
        data_name = "000"+str(i) 
    print("Getting Data from : {}".format(data_name))
    #Destination path
    if i<=15: 
      destination_lwir = os.path.join(destination_path,'A',"train" )
      destination_rgb = os.path.join(destination_path,'B',"train" )
      if i==15:
        print("Moving last subject in Training")
    elif (i>15 and i<=20):
      destination_lwir = os.path.join(destination_path,'A',"val" )
      destination_rgb = os.path.join(destination_path,'B',"val" )
      if i==20:
        print("Moving last subject in Val")
    else:
      destination_lwir = os.path.join(destination_path,'A',"test" )
      destination_rgb = os.path.join(destination_path,'B',"test" )

    data_path =  os.path.join(source_path, data_name) 

    # get homography transformation matrices
    H_rgb = np.array(np.load(os.path.join(data_path, "align_PTr_RGB.npy")))
    H_ir = np.array(np.load(os.path.join(data_path, "align_PTr_IR.npy")))
    H = np.matmul(np.linalg.inv(H_rgb), H_ir) # RGB -> IR tarnsformation
    tform = transform.ProjectiveTransform(matrix=H)


    for img_name in os.listdir(os.path.join(data_path,'RGB',cover_type)):
      if img_name.endswith('png'):
        
        #Source LWIR
        source = data_path  +'/IR/'+cover_type+'/'+img_name 
        shutil.copy(source, destination_lwir)
        os.rename(os.path.join(destination_lwir,img_name ),os.path.join(destination_lwir,str(count)+'.png' ))


        #Source RGB
        source = data_path  +'/RGB/'+cover_type+'/'+img_name 
        im_rgb = io.imread(source)
        im_rgb = transform.warp(im_rgb, tform)
        im_rgb = im_rgb[0:160, 0:120]
        io.imsave(os.path.join(destination_rgb,str(count)+'.png' ), img_as_ubyte(im_rgb))
        count+=1
        
  return count

In [ ]:
source_path = "/content/drive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/train/"
destination_path = "/content/drive/MyDrive/LWIR_to_RGB"
#get_data(source_path,destination_path, 'uncover',1,20)  ## Transfering uncovered images for train and val data
gtest_count = get_data(source_path,destination_path, 'cover1',35,37)  # Transfering thin covered images for test data
get_data(source_path,destination_path, 'cover2',76,77,gtest_count )  # Transfering thick covered images for test data

Getting Data from : 00035
Getting Data from : 00036
Getting Data from : 00037
Getting Data from : 00076
Getting Data from : 00077


226

# Install 
 from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

fatal: destination path 'pytorch-CycleGAN-and-pix2pix' already exists and is not an empty directory.


In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

# Create Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_pix2pix_dataset.sh [cityscapes, night2day, edges2handbags, edges2shoes, facades, maps]`

Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

In [ ]:
cd '/content/drive/MyDrive/LWIR_to_RGB/pytorch-CycleGAN-and-pix2pix'

/content/drive/.shortcut-targets-by-id/12Sz3Xm5GOveoM4c19uxdsdFReUqsU8dW/LWIR_to_RGB


Run this to create dataset used uncovered LWIR and RGB pairs in training and Validation, and covered pairs in test

In [ ]:
!python datasets/combine_A_and_B.py --fold_A /content/drive/MyDrive/LWIR_to_RGB/A  --fold_B /content/drive/MyDrive/LWIR_to_RGB/B --fold_AB /content/drive/MyDrive/LWIR_to_RGB

[fold_A] =  /content/drive/MyDrive/LWIR_to_RGB/A
[fold_B] =  /content/drive/MyDrive/LWIR_to_RGB/B
[fold_AB] =  /content/drive/MyDrive/LWIR_to_RGB
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = train, use 675/675 images
split = train, number of images = 675
split = val, use 225/225 images
split = val, number of images = 225
split = test, use 225/225 images
split = test, number of images = 225


#Training Pix2Pix

In [ ]:
cd '/content/drive/MyDrive/LWIR_to_RGB'

/content/drive/.shortcut-targets-by-id/12Sz3Xm5GOveoM4c19uxdsdFReUqsU8dW/LWIR_to_RGB


In [ ]:
!python /content/drive/MyDrive/LWIR_to_RGB/pytorch-CycleGAN-and-pix2pix/train.py --dataroot /content/drive/MyDrive/LWIR_to_RGB --name lwir_rgb_jat_3 --model pix2pix --direction AtoB  --save_epoch_freq 200 --n_epochs 100 --n_epochs_decay 100 

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/LWIR_to_RGB	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
          

In [ ]:
ls '/content/drive/MyDrive/LWIR_to_RGB/checkpoints/lwir_rgb_jat_3/'

latest_net_D.pth  latest_net_G.pth  loss_log.txt  train_opt.txt  web/


#Testing on trained model using covered images


In [ ]:
!python /content/drive/MyDrive/LWIR_to_RGB/pytorch-CycleGAN-and-pix2pix/test.py --dataroot /content/drive/MyDrive/LWIR_to_RGB  --direction AtoB --model pix2pix --name lwir_rgb_jat_3 --num_test 180

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/LWIR_to_RGB	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                        

# Not necessary but kept for reference

## Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

## Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

## Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)

## Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
!bash ./scripts/download_pix2pix_model.sh facades_label2photo

Note: available models are edges2shoes, sat2map, map2sat, facades_label2photo, and day2night
Specified [facades_label2photo]
for details.

--2021-06-08 15:05:45--  http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/facades_label2photo.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217704720 (208M)
Saving to: ‘./checkpoints/facades_label2photo_pretrained/latest_net_G.pth’

./checkpoints/facad 100%[===================>] 207.62M  1.17MB/s    in 96s     

2021-06-08 15:07:22 (2.17 MB/s) - ‘./checkpoints/facades_label2photo_pretrained/latest_net_G.pth’ saved [217704720/217704720]



In [ ]:
!python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./LWIR_to_RGB/A               	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              